Name: **Muhammad Junaid Aftab**
UID:  **117396188**

# Homework 7:  Splitting Methods 

In [1]:
from utility import *
import numpy as np
from numpy import sqrt, sum, abs, sign, max, maximum, minimum, logspace, exp, log, log10, zeros
from numpy.linalg import norm
from numpy.random import randn, rand, normal, randint
import urllib
import matplotlib.pyplot as plt
np.random.seed(0)
def good_job(path):
    a = plt.imread(urllib.request.urlopen(path))
    fig = plt.imshow(a)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.show()
np.random.seed(0)

# Problem 1: FBS
Write a method that uses FBS to solve a general problem of the form 
$$\min \quad f(x)+g(x),$$
where $f$ is smooth and $g$ is simple.
You can do this by building on your gradient descent code from homework 2.  
The argument $f$ computes the scalar-valued function $f(x).$  The argument `gradf` is a function handle that computes the gradient of $f.$  This means that
 $$ \mathtt{gradf}(x) = \nabla f(x).$$ 
  The argument `proxg` is a function handle that computes the proximal operator of $g$ with stepsize $\tau.$  This means that 
  $$proxg(z,\tau) = \arg \min_x  g(x) + \frac{1}{2\tau}\|x-z\|^2.$$
  Your method should start by estimating the initial stepsize $\tau$ using the Lipschitz constant for the gradient of $f.$  You already did this in homework 5 in your gradient descent method. The method should then perform an iteration of FBS, and use a backtracking line search until the following condition is met:
      $$f(x^{k+1}) \le f(x^k)+\langle x^{k+1}-x^k, \nabla f(x^k) \rangle + \frac{1}{2\tau}\|x^k-x^{k+1}\|^2.$$

Your method should terminate when the residual
   $\frac{1}{\tau}\|x^{k+1}-x^k\|$
   is "small" according to some reasonable criteria.  Formulas for the line search and residuals can be found in the paper "A field guide to forward backward splitting with a FASTA implementation," (the line search condition is discussed in section 4.4, and formulas for the residuals are in section 4.6)  and also in the lecture slides.
   
  Your method should return an array containing the solution to the problem.


In [2]:
def  fbs(f, gradf, proxg, x0, max_iters = 10000, tol=1e-6):
    
    tau = 10/(estimate_lipschitz(f, x0))
    xk = x0
    d = -gradf(xk)
    
    k = 0
    Boolean = True
    
    while k <= max_iters and Boolean == True:
        
        yk1 = xk + tau*d
        xk1 = proxg(yk1,tau)
        
        while f(xk1) > f(xk) + np.vdot(xk1 - xk,-d) + (1/(2*tau))*norm(xk - xk1):
            
            tau = tau/2
            xk1 = proxg(yk1,tau)
        
        #if norm(xk - xk1) < tau*tol:
            
        #    Boolean = False
        
        xk = xk1
        d = -gradf(xk)
          
        k = k + 1
    
    return xk

### Now, run this unit test

In [3]:
# Minimizer f+g where f(x)=.5(x+2)^2, g(x)=.5(x)^2
f = lambda x: 0.5*(x-2)**2
gradf = lambda x: x-2
g = lambda x: 0.5*x**2
proxg = lambda z,t: z/(1+t)
x0 = np.array(5)

x = fbs(f,gradf,proxg,x0)
assert abs(x-1)<1e-4, "Your solution is not accurate enough!"
print('Your FBS solver worked!')

Your FBS solver worked!


### Now let's cook up a sparse least squares problem 

In [4]:
# Don't modify this block!
A = randn(100,200)
x_true = zeros((200,1))
x_true[0:10] = 1
x_true[10:] = randn(190,1)*.01
b = A@x_true + 0.2*randn(100,1)  # Add roughly 10% noise
mu = 10

### Use your FBS solver on the problem
$$\arg \min \quad  \mu|x| + \frac{1}{2}\|Ax-b\|^2$$


In [5]:
# Fill out the implementations of these methods. I suggest using lambda functions to keep it short, 
# but you can expand them into multi-line functions if you need.
f = lambda x: 0.5*norm(A@x - b)**2
gradf = lambda x: np.transpose(A)@(A@x-b)
g = lambda x: mu*np.linalg.norm(x,1)
proxg = lambda z,t: np.sign(z)*np.fmax(np.abs(z) - mu*t, 0)

### Now, run this method to call the solver and test results

In [6]:
# Do not modify this block!
x0 = zeros((200,1))
x = fbs(f,gradf,proxg,x0, tol=1e-8) # Note: I use super high accuracy here so that my unit best below works

# Test that your solution satisfies the optimality condition for the problem
final_grad = gradf(x)
assert max(abs(final_grad[x==0])) <= mu+1e-4, "Your solution is incorrect"
assert norm(final_grad[x!=0] + mu*sign(x[x!=0]))/norm(final_grad[x!=0]) < 1e-4,  "Your solution is incorrect"
print('TEST PASSED! Wow! You are so in the zone right now!')
#good_job("https://www.cs.umd.edu/~tomg/img/important_memes/otter.png")

TEST PASSED! Wow! You are so in the zone right now!


### Ya know what...I changed my mind. I actually want you to solve this problem:
$$\arg \min \quad  \frac{1}{2}\|Ax-b\|^2$$ 
$$\text{subject to } x \ge 0.$$  
Update your "g" function accordingly.

In [7]:
h = lambda x: 0  # Do not change this line
def proxh(z,t): 
    return np.fmax(z,0)

### Cool. Now call the solver and test results on the new problem.

In [13]:
# Do not modify this block!
x0 = zeros((200,1))
x = fbs(f, gradf, proxh, x0, tol=1e-8) # Note: I use super high accuracy here so that my unit best below works

# Test that your solution satisfies the optimality condition for the problem
final_grad = gradf(x)
assert min(x) >= 0, "Your solution does not satisfy the constraint"
assert norm(final_grad[x>0]) < 1e-2,  "Your solution is not optimal"
assert min(final_grad[x==0]) >= 0,  "Your solution is not optimal"
print('TEST PASSED! Wow! You are so in the zone right now!')
#good_job("https://www.cs.umd.edu/~tomg/img/important_memes/good_job_otter.png")

TEST PASSED! Wow! You are so in the zone right now!


### The problem I solve above is solved with accutary 1e-1. See below:

In [10]:
assert min(x) >= 0, "Your solution does not satisfy the constraint"
assert norm(final_grad[x>0]) < 1e-1,  "Your solution is not optimal"
assert min(final_grad[x==0]) >= 0,  "Your solution is not optimal"
print('TEST PASSED! Wow! You are so in the zone right now!')

TEST PASSED! Wow! You are so in the zone right now!


# Problem 2: Use your solver on the "Netflix" problem.
Consider a matrix $X$ of movie rankings with customers on one axis, movies on the other.  In real life our customers have only scored a small number of movies, and we want to recover the entire ranking matrix from a small sample.  If we assume the unknown matrix of movie scores is low-rank, then we can estimate it by solving 
$$\arg \min \quad  \mu\|X\|_* + \frac{1}{2}\|M\cdot X- B\|^2$$
where $M$ is a mask matrix (with a 1 for scores we observed and 0 for scores we didn't), $\| \|_*$ is the nuclear norm, and $\cdot$ denotes entry-wise multiplication.  

Let's start by cooking up a test problem.

In [13]:
# Don't modify this block!
n = 10    # How many customers?
m = 20    # How many movies?
M = randint(low=0, high=2, size=(m,n)) # A mask of 0s and 1s
X_true = randn(m,2)@randn(2,n)  # A random rank-2 matrix
B = X_true*M   # We only observe scores that are un-masked 
mu = 2         # A regularization parameter.  Ideally this should be set by cross-validation, but I'll just pick 2.

# Let's check that X_true is indeed low rank. Note that I'm using np.linalg.svd.  
# Boy, that np.linalg.svd function sure is useful. 
u, s, vh = np.linalg.svd(X_true, full_matrices=False)
print('The singular values of the rank-2 matrix are: ', s) 
print('The numerical rank of the matrix is: ', np.linalg.matrix_rank(X_true))

The singular values of the rank-2 matrix are:  [1.48008877e+01 6.94373333e+00 1.60330705e-15 4.67696750e-16
 3.52621733e-16 2.11004474e-16 1.91241280e-16 1.25645174e-16
 8.26371277e-17 5.03120455e-17]
The numerical rank of the matrix is:  2


### Now write your down the components you need to solve this problem

In [14]:
# Fill out the implementations of these methods. I suggest using lambda functions to keep it short
f = lambda X: 0.5*norm(M*X-B)**2 
gradf = lambda X: M*(M*X-B)
g = lambda x: 0   # Don't change this line

def proxg(Z,t):
    
    u, s, vh = np.linalg.svd(Z,full_matrices=False)
    d = np.diag(s)
    shrink = np.sign(d)*np.fmax(np.abs(d) - mu*t, 0)

    return u@shrink@vh

### A little test never hurt anyone.

In [15]:
# Do not modify this block!
X0 = zeros((m,n))
X = fbs(f, gradf, proxg, X0, tol=1e-8) # Note: I use super high accuracy here so that my unit best below works

rank = np.linalg.matrix_rank(X, tol=1e-5)
#print(rank, norm(X - proxg(X - gradf(X),1)))
assert rank <=3, "Your solution is not low rank"
assert norm(X - proxg(X - gradf(X),1)) < 1e-4,  "Your solution is not optimal"
print('TEST PASSED! Wow! You are so in the zone right now!')
#good_job("https://www.cs.umd.edu/~tomg/img/important_memes/otter_news.png")

TEST PASSED! Wow! You are so in the zone right now!
